<h1> Data Collection </h1>

<h2> Importing libraries, setting up drivers, opening the site </h2>

In [ ]:
from selenium import webdriver
from datetime import datetime
import csv

In [ ]:
#Path to Chrome Driver for windows
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#Open site
driver.get("https://stockx.com/sneakers")

#Click on the annoying pop-up
driver.implicitly_wait(10)
PopUpQuitButton = driver.find_element_by_xpath("//img[@src='//stockx-assets.imgix.net/AllInAsk/close (1).svg?auto=compress,format']").click()

<h2> Functions </h2>

<h3> Functions for avg price collection </h3>

In [ ]:
#Function that takes in date array and price array and returns the average price for the sale history of previous months
def get_avg_new_price(date_arr, price_arr):
    #Array for new prices
    new_price_arr = []

    #Find timestamp for last 2 month
    now = datetime.now().timestamp()
    month_seconds = 2628288
    last_month_ts = now - (month_seconds*2)

    #Filter out and append new prices to the array
    for i in range(len(date_arr)):
        if date_arr[i] > last_month_ts:
            new_price_arr.append(price_arr[i])

    #Averaging all values in new_price_arr
    average_price = sum(new_price_arr)/len(new_price_arr)
    return round(average_price,2)

def get_price():
    with open("Tableinfo.txt","r") as f:
        lines =f.readlines()
        for line in lines:
            if "$" in line:
                LineList = line.split("$")
                price = LineList[1].strip()

def extract_date(dateString):
    #Remove "Sunday,"
    dateString = ",".join(dateString.split(",")[1:]).strip()
    datetimeObj = datetime.strptime(dateString, "%B %d, %Y %I:%M %p")
    datetime_timestamp = datetimeObj.timestamp()
    return datetime_timestamp

<h3> Functions for getting numpy array </h3>

In [1]:
#Function that takes in special name array of sneaker and returns numpy array
def GetNumpyArray(SpecificNameArr):

    #Returns an array of sneaker names from numpysneakernames.txt in specific order
    SneakerNamesArr = []

    with open("NumpySneakerNames.txt", "r") as f:
        lines = f.readlines()
        for line in lines:
            SneakerName = line.strip("\n")
            SneakerNamesArr.append(SneakerName)

    #Sneaker name arr for testing
    TrueCount = 0 #Counter
    NumpyArr = [] #Numpy array for sneaker

    #For loop to check if the value is true in sneaker name arr, returns 1 if true and 0 if false
    for i in range(len(SneakerNamesArr)):
        if SpecificNameArr[TrueCount] == SneakerNamesArr[i]:
            NumpyArr.append(1)
            if TrueCount != len(SpecificNameArr)-1:
                TrueCount = TrueCount + 1
        else:
            NumpyArr.append(0)

    return NumpyArr

#Functions to split the sneaker name into special name arrays
def strip(s):
    return s.strip()

def splitSpecial(sneakerName):
    #iterate through specialNames
    for s in SpecialNamesArr:
        #check if special name is in sneaker name
        if s in sneakerName:
            nameArray = sneakerName.split(s)
            nameArray.insert(1, s)
            nameArray = list(map(strip, nameArray))

            #Check for space in 3rd element and split if there is space
            if "" in nameArray[2]:
                element2 = nameArray[2]
                del nameArray[2]
                newArr2 = element2.split()
                for i in range(len(newArr2)):
                    position2 = 2+i
                    value2 = newArr2[i]
                    nameArray.insert(position2,value2)

            #Check for space in 1st element and split if there is space
            if "" in nameArray[0]:
                element1 = nameArray[0]
                del nameArray[0]
                newArr1 = element1.split()
                for i in range(len(newArr1)):
                    position1 = i
                    value1 = newArr1[i]
                    nameArray.insert(position1,value1)

            return nameArray
    return sneakerName

<h2> Code to run task of getting data and processing data to store </h2>

In [ ]:
#For loop to loop through all 40 pairs on the first page
for i in range(1):
    
    #Click on the i + 1 sneaker
    xpathstr = "//div[@class='tile browse-tile']["+str(i+1)+"]"
    SneakerSelect = driver.find_element_by_xpath(xpathstr).click()

    #Click on button to view all sales of the sneaker
    driver.find_element_by_xpath("//a[@data-testid='product-viewallsales']").click()

    #Loop to press load more button to load full table. It will press load more button 10 times and then it will break out of loop. If there is not up to 10 button presses, it will automatically leave the loop. This should allow a little over 100 of the sale history.
    press_count = 0
    while True:
        try:
            load_more_button = driver.find_element_by_xpath("//button[@class='button button-block button-white']").click()
            press_count += 1
            if press_count == 10:
                break
        except:
            break
    
    #Find element for the sale history table and 
    table = driver.find_element_by_xpath("//table[@id='480']").text

    #Clears table info file to make sure it is empty
    with open("TableTempinfo.txt", "r+") as f:
        f.truncate(0)

    #Open table file and overwrite the file with the new data
    with open ("TableTempinfo.txt", "w") as f:
        f.write(table)

#========== Getting avg price and storing to csv file ==========#

    #Extract price and date and append to arrays
    price_arr = []
    date_arr = []

    #Extract price and date from Tableinfo.txt file
    with open("TableTempinfo.txt","r") as f:
        lines =f.readlines()
        
        for line in lines:
            
            if "$" in line:
                LineList = line.split("$")

                #Get price and store it into an array of integers
                price = int(LineList[1].replace(',','').strip())
                price_arr.append(price)

                #Split first half of line to get the date and the size
                date_and_size = LineList[0]
                date_size_split_list = date_and_size.split("EST")

                #Format date into timestamp and append to array
                date_string = date_size_split_list[0]
                formatted_date = extract_date(date_string)
                date_arr.append(formatted_date)

    #Finding average for sales in previous month
    avg_price = get_avg_new_price(date_arr, price_arr)

    #Append average price to csv file
    with open('AveragePriceData.csv', 'a') as csvfile:
        writer = csv.writer(csvfile, lineterminator = '\r')
        writer.writerow([avg_price])

    #Get product name and display message of the sneaker name and the average price
    product_name = driver.find_element_by_xpath("//h1[@data-testid='product-name']").text
    print("Average price for " + str(product_name) + " is $" + str(avg_price))

    #Clear arrays and reset all counters, ready for the next pair
    price_arr.clear()
    date_arr.clear()

#========== Getting numpy array for sneaker name ==========#

    SneakerName = driver.find_element_by_xpath("//h1[@data-testid='product-name']").text

    SpecialNameResultArr = splitSpecial(SneakerName)

    #Return to initial page and refresh
    driver.back()
    driver.refresh()